# H_22082024
# Analisis del volumen profile para encontrar zonas de baja cotización (FVG like) y como se comporta el precio en esas zonas.

En este Python Notebook analizaremos, con datos de mt5, como se comporta el mercado en las zonas previamente definidas.

## Conceptos clave del perfil de mercado:

1. **Oportunidad de precio en el tiempo (TPO):**
    
    representa los niveles de precio negociados durante intervalos de tiempo específicos. En un gráfico de perfil de mercado, las TPO indican la frecuencia con la que se negoció cada nivel de precio durante una sesión.
    
2. **Área de valor:**
    
    rango de precios en el que se produjo el 70 % de la actividad comercial durante una sesión. Esta área representa el valor justo percibido del mercado.
    
3. **Punto de control (POC):**
    
    el nivel de precio con el mayor volumen durante la sesión. El POC suele ser un nivel clave de soporte o resistencia.
    
4. **Balance inicial (IB):**
    
    rango de precios negociados durante la primera hora de la sesión de negociación. Proporciona una visión anticipada de la estructura del mercado del día.

In [36]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
import json
import plotly.express as px
import plotly.graph_objects as go

## 1. Definición de zonas de baja cotización para posterior analisis.
El primer paso a dar es definir como encontraremos las **zonas de baja cotización** en una sesion de mercado de forma objetiva.

A continuación, definiremos que es una zona de baja cotización para nuestro estudio:

1. Analisis basado en el *volumen profile*, el cual analiza el numero de cotizaciónes por precio que tiene lugar en una sesión de trading predefinida (en nuestro estudio, una sesión será un dia 1D).
2. La media y la StdDev del volumen profile de la sesión nos ayudarán a analizar correctamente las zonas.

Por tanto, una **zona de baja cotización** consiste en:
- Los niveles de precio que se encuentran N*StdDev(mean(volume_profile)) por debajo.

In [37]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# request connection status and parameters
print(mt5.terminal_info())
# get data on MetaTrader 5 version
print(mt5.version())

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")
# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2024, 6, 10, tzinfo=timezone)
utc_to = datetime(2024, 6, 11, tzinfo=timezone)

# request AUDUSD ticks within 11.01.2020 - 11.01.2020
ohlcv = mt5.copy_rates_range("GC_V", mt5.TIMEFRAME_M1, utc_from, utc_to)

mt5.shutdown()

TerminalInfo(community_account=True, community_connection=True, connected=True, dlls_allowed=True, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=4410, maxbars=100000, codepage=1252, ping_last=45856, community_balance=0.0, retransmission=0.9587199302308679, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='Spanish', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\iamfr\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\iamfr\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
(500, 4410, '21 Jun 2024')


True

In [48]:
# convert time in seconds into the datetime format
df = pd.DataFrame(ohlcv)
df['time']=pd.to_datetime(df['time'], unit='s')

df = df.set_index('time')
del df['real_volume']
del df['spread']

# display data
print("\nDisplay dataframe with ticks")
print(df.head(10)) 


Display dataframe with ticks
                       open    high     low   close  tick_volume
time                                                            
2024-06-10 02:00:00  2338.3  2338.8  2338.1  2338.2          100
2024-06-10 02:01:00  2338.1  2338.4  2337.8  2338.4           74
2024-06-10 02:02:00  2338.3  2338.8  2338.3  2338.7           76
2024-06-10 02:03:00  2338.8  2339.0  2338.6  2338.9           59
2024-06-10 02:04:00  2338.8  2339.5  2338.5  2339.3          113
2024-06-10 02:05:00  2339.2  2339.3  2338.9  2339.0           77
2024-06-10 02:06:00  2338.9  2338.9  2338.3  2338.3          117
2024-06-10 02:07:00  2338.4  2338.6  2338.1  2338.3           74
2024-06-10 02:08:00  2338.2  2338.4  2338.1  2338.4           52
2024-06-10 02:09:00  2338.3  2338.3  2338.1  2338.2           42


In [49]:
close = df['close']
volume = df['tick_volume']
# px.histogram(df, x='tick_volume', y='close', nbins=150, orientation='h').show()

fig = px.histogram(df, x='tick_volume', y='close')
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed